## Get subscription

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime
from azure.mgmt.subscription import SubscriptionClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
sub_client = SubscriptionClient(credential)
subscription = next(sub_client.subscriptions.list(), None)
if not subscription:
    raise Exception("Authenticate using the az cli")
subscriptionId = subscription.subscription_id
print(f"Using subscription {subscriptionId}")

## Getting regions

In [ ]:
token = credential.get_token('https://management.azure.com/.default')
headers = {'Authorization': 'Bearer ' + token.token}

unique_region_names = set()

locations_request = f"https://management.azure.com/subscriptions/{subscriptionId}/locations?api-version=2021-04-01"

response = requests.get(locations_request, headers=headers)

data = json.loads(response.text)

for item in data["value"]:
    unique_region_names.add(item["name"])

regions_list = list(unique_region_names)

print(regions_list)

## Find regions where Azure AI is supported

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

regions_successful = []
regions_failed = []

def get_models(region):
    url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/models?api-version=2023-05-01"
    return (region, requests.get(url, headers=headers))

with tqdm(total=len(regions_list)) as pbar:
    with ThreadPoolExecutor(max_workers=len(regions_list)) as ex:
        futures = [ex.submit(get_models, region) for region in regions_list]
        for future in as_completed(futures):
            (region, result) = future.result()
            if result.status_code == 200:
                regions_successful.append(region)
            else:
                regions_failed.append(region)

            pbar.update(1)

print("Potential Azure OpenAI regions based on control plane response:")
print(regions_successful)

print("Azure OpenAI Not Supported:")
print(regions_failed)

In [ ]:
region_model_data = {}  

excluded_models =  ['text-similarity-ada-001', 'text-babbage-001', 'text-curie-001', 'text-similarity-curie-001', 'text-davinci-002','text-davinci-003', 'text-davinci-fine-tune-002', 'code-davinci-002', 'code-davinci-fine-tune-002','text-ada-001', 'text-search-ada-doc-001', 'text-search-ada-query-001', 'code-search-ada-code-001','code-search-ada-text-001', 'text-similarity-babbage-001', 'text-search-babbage-doc-001','text-search-babbage-query-001', 'code-search-babbage-code-001', 'code-search-babbage-text-001', 'text-search-curie-doc-001', 'text-search-curie-query-001', 'text-davinci-001','text-similarity-davinci-001', 'text-search-davinci-doc-001', 'text-search-davinci-query-001','code-cushman-001']

for region in regions_successful:
    model_request_url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/models?api-version=2023-05-01"
    model_response = requests.get(model_request_url, headers=headers)

    response_dict = json.loads(model_response.text)
    #print(response_dict)
    data_test = []

    for item in response_dict["value"]:
        model_name = None
        version = None
        sku_name = None
        if item["model"]["capabilities"].get("scaleType") == "Manual": #skip legacy models
            continue
        model_name = item["model"]["name"]
        if model_name in excluded_models: # if in list skip
            continue
        version = item["model"]["version"]
        rdate = item["model"]["deprecation"]
        for sku in item["model"]["skus"]:
            sku_name = sku["name"]
        if sku_name == "Standard": # This example is only targeting Standard Model deployments SKUI
            data_test.append({"Model Name": model_name, "Version": version, "SKU Name": sku_name})
                #print(data_test)

    region_model_data[region] = data_test  # store the model data under corresponding region name

# Print result
for region, model_data in region_model_data.items():
    print(f'{region}: {model_data}')


            

In [ ]:
rows = []
for region, models in region_model_data.items():
    for model in models:
        row = model.copy()  
        row['Region'] = region  
        rows.append(row)

df = pd.DataFrame(rows)
df = df[['Region', 'Model Name', 'Version', 'SKU Name']]
pd.set_option('display.max_rows', None)

df['Exist'] = True 
pivot_df = df.pivot_table(index='Region', columns=['Model Name', 'Version'], values='Exist', fill_value=False, aggfunc='any')
pivot_df.reset_index(inplace = True)

pivot_df

In [ ]:
region_model_quota_data = {}

# List of models to exclude
exclude_models = ["Code-Cushman-001", "code-cushman-fine-tune-002", "Code-Search-Ada-Code-001", "Code-Search-Ada-Text-001", "Text-Ada-001", "Text-Search-Ada-Doc-001", "Text-Search-Ada-Query-001", "Text-Similarity-Ada-001", "Babbage", "Code-Search-Babbage-Code-001", "Code-Search-Babbage-Text-001", "Text-Babbage-001", "Text-Search-Babbage-Doc-001", "Text-Search-Babbage-Query-001", "Text-Similarity-Babbage-001", "Curie", "Text-Curie-001", "Text-Search-Curie-Doc-001", "Text-Search-Curie-Query-001", "Text-Similarity-Curie-001", "Code-Davinci-002", "Code-Davinci-Fine-Tune-002", "Davinci", "Text-Davinci-001", "Text-Davinci-002", "Text-Davinci-003", "Text-Davinci-Fine-Tune-002", "Text-Search-Davinci-Doc-001", "Text-Search-Davinci-Query-001", "Text-Similarity-Davinci-001", "Code-Cushman-001", "code-cushman-fine-tune-002",  "Ada", "Code-Search-Ada-Code-001", "Code-Search-Ada-Text-001", "Text-Ada-001", "Text-Search-Ada-Doc-001", "Text-Search-Ada-Query-001", "Text-Similarity-Ada-001", "Babbage", "Code-Search-Babbage-Code-001", "Code-Search-Babbage-Text-001", "Text-Babbage-001", "Text-Search-Babbage-Doc-001", "Text-Search-Babbage-Query-001", "Text-Similarity-Babbage-001", "Curie", "Text-Curie-001", "Text-Search-Curie-Doc-001", "Text-Search-Curie-Query-001", "Text-Similarity-Curie-001", "Code-Davinci-Fine-Tune-002", "Davinci", "Text-Davinci-001", "Text-Davinci-002", "Text-Davinci-003", "Text-Davinci-Fine-Tune-002", "Text-Search-Davinci-Doc-001", "Text-Search-Davinci-Query-001", "Text-Similarity-Davinci-001"]

for region in regions_successful:
    url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/usages?api-version=2023-05-01"
    response = requests.get(url, headers=headers)
    response_dict = json.loads(response.text)

    token_limit_details = []

    
    for item in response_dict['value']:
        # Ignoring 'OpenAI.LowPriority.gpt-4' value
        if item['name']['value'] == 'OpenAI.LowPriority.gpt-4':
            continue

        # Check if 'Tokens Per Minute' is in the localized value
        if 'Tokens Per Minute' in item['name']['localizedValue']:
        # Extract the localized value & limit.
            localized_value = item['name']['localizedValue']
            localized_value = localized_value.replace("Tokens Per Minute (thousands) - ", "")
            limit = item['limit']
            #print(localized_value)
            #print(limit)

            # If the model is not in the exclude list, add it to the token_limit_details
            if localized_value not in exclude_models:
                token_limit_details.append((localized_value, limit))
            
        elif 'Enqueued tokens' in item['name']['localizedValue']:
        # Extract the localized value & limit.
            localized_value = item['name']['localizedValue']
            #print(localized_value)
            localized_value = localized_value.replace("Enqueued tokens (thousands) - ", "")
            localized_value = localized_value + " - Global-Batch"
            #print(localized_value)
            limit = item['limit']
            #print(limit)
        
        # If the model is not in the exclude list, add it to the token_limit_details
            if localized_value not in exclude_models:
                token_limit_details.append((localized_value, limit))
            
    region_model_quota_data[region] = token_limit_details

print(region_model_quota_data)

In [ ]:
flattened_data = [(region, lv, limit) for region in region_model_quota_data for lv, limit in region_model_quota_data[region]]

    ## Convert the list to a DataFrame
df = pd.DataFrame(flattened_data, columns=['Region', 'LocalizedValue', 'Limit'])
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)
 
    #print(df) 
pivot_df = df.pivot(index='Region', columns='LocalizedValue', values='Limit')
pivot_df.reset_index(level=0, inplace=True)


    # Specify the base column ordering you want
column_order = ["Region", "o1-mini", "o1", "gpt-4o", "gpt-4o-mini", "GPT-4", "GPT-4-32K", "GPT-4-Turbo", "GPT-4-Turbo-V", "GPT-35-Turbo", "GPT-35-Turbo-Instruct", "o1-mini - GlobalStandard",  "o1 - GlobalStandard", "gpt-4o - GlobalStandard", "gpt-4o-mini - GlobalStandard", "GPT-4-Turbo - GlobalStandard", "GPT-4o - Global-Batch", "GPT-4o-mini - Global-Batch", "GPT-4 - Global-Batch", "GPT-4-Turbo - Global-Batch", "gpt-35-turbo - Global-Batch", "Text-Embedding-Ada-002", "text-embedding-3-small", "text-embedding-3-large", "GPT-4o - finetune", "GPT-4o-mini - finetune", "GPT-4 - finetune",  "Babbage-002", "Babbage-002 - finetune","Davinci-002", "Davinci-002 - finetune","GPT-35-Turbo - finetune", "GPT-35-Turbo-1106 - finetune"]

# Create a set of column names from column_order and the current DataFrame
set_column_order = set(column_order)
set_existing_columns = set(pivot_df.columns)

# New columns which are not in the column_order list
new_columns = list(set_existing_columns - set_column_order)

# Add the new columns to the end of the column_order list
column_order = column_order + new_columns

# Reorder the dataframe
pivot_df = pivot_df[column_order]

pivot_df.columns.name = None
pivot_df.fillna(-1, inplace=True)
numeric_columns = pivot_df.columns.drop('Region') #Get all numeric columns

pivot_df[numeric_columns] = pivot_df[numeric_columns].astype(int)
pivot_df[numeric_columns] = pivot_df[numeric_columns].applymap(lambda x: str(x) if x != -1 else '-')



pivot_df